In [0]:
# CNN in pytorch as raw as you can
# Goals resnet18, accuracy=0.968597, using only nn.sequential, build blocks

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai.vision import *

## Data

In [4]:
path = untar_data(URLs.MNIST_SAMPLE)
path

PosixPath('/root/.fastai/data/mnist_sample')

In [0]:
data = ImageDataBunch.from_folder(path) # batch size=64

In [6]:
data # mnist sample has 3 dimensions

ImageDataBunch;

Train: LabelList (12396 items)
x: ImageList
Image (3, 28, 28),Image (3, 28, 28),Image (3, 28, 28),Image (3, 28, 28),Image (3, 28, 28)
y: CategoryList
3,3,3,3,3
Path: /root/.fastai/data/mnist_sample;

Valid: LabelList (2038 items)
x: ImageList
Image (3, 28, 28),Image (3, 28, 28),Image (3, 28, 28),Image (3, 28, 28),Image (3, 28, 28)
y: CategoryList
3,3,3,3,3
Path: /root/.fastai/data/mnist_sample;

Test: None

In [11]:
data.x

ImageList (12396 items)
Image (3, 28, 28),Image (3, 28, 28),Image (3, 28, 28),Image (3, 28, 28),Image (3, 28, 28)
Path: /root/.fastai/data/mnist_sample

In [12]:
# z = data.x[0:64]
# z
# # 3 dimensions are equal
# (z[0] == z[2]).sum() # 28x28

ImageList (64 items)
Image (3, 28, 28),Image (3, 28, 28),Image (3, 28, 28),Image (3, 28, 28),Image (3, 28, 28)
Path: /root/.fastai/data/mnist_sample

## Recreate ResNet18

In [0]:
m = nn.Sequential(
        nn.Conv2d(64, 128, kernel_size=3, stride=2,padding=1, bias=False)
        , nn.BatchNorm2d(128)
        , nn.ReLU(inplace=True)
        , nn.Conv2d(128, 128, kernel_size=3, stride=1,padding=1, bias=False)
        , nn.BatchNorm2d(128)
)

In [0]:
# Sense check sizes of input output in sequential models
input = torch.randn(64, 3, 28, 28)
# input = torch.randn(64, 64, 7, 7)
output = model(input)
# output = m(input)

In [62]:
output.size()

torch.Size([64, 2])

In [0]:
input.view(64,-1).size()
# torch.flatten(input, 1).size()

torch.Size([64, 512])

In [0]:
# source: https://stackoverflow.com/questions/53953460/how-to-flatten-input-in-nn-sequential-in-pytorch
class Flatten(torch.nn.Module):
    def forward(self, x):
        return torch.flatten(x, 1)
#         z = torch.flatten(x, 1)
#         return torch.cat((z,z), dim = 1) # 1024 in end layer even if only 512?

In [0]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=3, stride=stride,padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1,padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [0]:
downsampleL2 = nn.Sequential(
    nn.Conv2d(64, 128*1, kernel_size=1, stride=2, bias=False)
    ,nn.BatchNorm2d(128*1)
            )

In [0]:
downsampleL3 = nn.Sequential(
    nn.Conv2d(128, 256*1, kernel_size=1, stride=2, bias=False)
    ,nn.BatchNorm2d(256*1)
            )

In [0]:
downsampleL4 = nn.Sequential(
    nn.Conv2d(256, 512*1, kernel_size=1, stride=2, bias=False)
    ,nn.BatchNorm2d(512*1)
            )

In [0]:
class AdaptiveConcatPool2d(Module):
    "Layer that concats `AdaptiveAvgPool2d` and `AdaptiveMaxPool2d`."
    def __init__(self, sz:Optional[int]=None):
        "Output will be 2*sz or 2 if sz is None"
        self.output_size = sz or 1
        self.ap = nn.AdaptiveAvgPool2d(self.output_size)
        self.mp = nn.AdaptiveMaxPool2d(self.output_size)

    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)

In [0]:
# Try using basic blocks for identity connections + compressed code
# Resnet 18: https://arxiv.org/abs/1502.01852 Model A, Table 3 Page7
model = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,bias=False)
    , nn.BatchNorm2d(64)
    , nn.ReLU(inplace=True)
    , nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    
    # Layer 1
    , BasicBlock(64, 64, stride=1, downsample=None) # Block 1
    , BasicBlock(64, 64, stride=1, downsample=None) # Block 2
    # Layer 2
    , BasicBlock(64, 128, stride=2, downsample = downsampleL2) # Block 1
    , BasicBlock(128, 128, stride=1, downsample = None) # Block 2
    # Layer 3
    , BasicBlock(128, 256, stride=2, downsample = downsampleL3) # Block 1
    , BasicBlock(256, 256, stride=1, downsample = None) # Block 2
    # Layer 4
    , BasicBlock(256, 512, stride=2, downsample = downsampleL4) # Block 1
    , BasicBlock(512, 512, stride=1, downsample = None) # Block 2
    
    , AdaptiveConcatPool2d(1) # Custom fastai method
#     , nn.Linear(in_features=512, out_features=1000, bias=True) # original output layer
    
    # Additional layers to adapt to any dataset
    # Should be 1024 {
    , Flatten()
    , nn.BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    , nn.Dropout()
    # }
    , nn.Linear(in_features=1024, out_features=512, bias=True)
    , nn.ReLU(inplace=True)
    , nn.BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    , nn.Dropout()
    , nn.Linear(in_features=512, out_features=2, bias=True) # <-- edit out_features to number of classes
)

In [0]:
learn = Learner(data, model, loss_func = nn.CrossEntropyLoss(), metrics=accuracy)

In [67]:
# your model
learn.summary()

Sequential
Layer (type)         Output Shape         Param #    Trainable 
Conv2d               [64, 14, 14]         9,408      True      
______________________________________________________________________
BatchNorm2d          [64, 14, 14]         128        True      
______________________________________________________________________
ReLU                 [64, 14, 14]         0          False     
______________________________________________________________________
MaxPool2d            [64, 7, 7]           0          False     
______________________________________________________________________
Conv2d               [64, 7, 7]           36,864     True      
______________________________________________________________________
BatchNorm2d          [64, 7, 7]           128        True      
______________________________________________________________________
ReLU                 [64, 7, 7]           0          False     
___________________________________________________

In [0]:
learn.lr_find(end_lr=100)
learn.recorder.plot()

In [0]:
learn.fit_one_cycle(3, max_lr=0.05)

## Goal model architecture

In [0]:
# correct model for MNIST sample
learn = cnn_learner(data, models.resnet18, metrics=accuracy)
learn.summary()

Sequential
Layer (type)         Output Shape         Param #    Trainable 
Conv2d               [64, 14, 14]         9,408      False     
______________________________________________________________________
BatchNorm2d          [64, 14, 14]         128        True      
______________________________________________________________________
ReLU                 [64, 14, 14]         0          False     
______________________________________________________________________
MaxPool2d            [64, 7, 7]           0          False     
______________________________________________________________________
Conv2d               [64, 7, 7]           36,864     False     
______________________________________________________________________
BatchNorm2d          [64, 7, 7]           128        True      
______________________________________________________________________
ReLU                 [64, 7, 7]           0          False     
___________________________________________________

In [0]:
# out of the box resnet18
models.resnet18()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

## fin